In [2]:
#필요 라이브러리 설치
#!pip install selenium 

#selenium webdriver 관련 모듈
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup #크롤링한 정보를 파싱하고 정리하기 위한 모듈.
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm #시간별 진행상황을 보기 위한 모듈
import re #정규식
import os
from urllib import request
from urllib.error import HTTPError
import time 
from time import sleep
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings("ignore")
import random

In [3]:
def read_file_to_list(file_path):
    df=pd.read_csv(file_path)
    columns = df[['Title','Artist']]
    combined_list = [','.join(row) for row in columns.values]

    return combined_list, df['Track ID']

file_path="../data/filtered_lyrics_with_sets.csv"
list_content, song_ids = read_file_to_list(file_path)

Id_keyword = pd.DataFrame({
    'Track ID': song_ids,
    'Title Artist': list_content

})

print(Id_keyword)


                Track ID                                      Title Artist
0     TRMMATD128F932C32B  You Were Alone,Casiotone For The Painfully Alone
1     TRMMKAR128EF3431B1                                   Without You,Ayo
2     TRMWQYD128F426639B                  Pony Blues,Alvin Youngblood Hart
3     TRMWXEM128F429D851          Do You Wanna Dance,The Mamas & The Papas
4     TRMGNHR128F428B17E                 Since K Got Over Me,The Clientele
...                  ...                                               ...
1662  TRYYCGL12903CB11C2                       Danville Girl,Woody Guthrie
1663  TRYYPVU12903CCE5A6                Sweeter the Victory,Gregory Isaacs
1664  TRYYTVV12903D03F75               I Wish It Would Rain,RODNEY CROWELL
1665  TRYYJQW12903CECBC9                              A Miner's Life,Solas
1666  TRYYDBK128F930FDBC                        Under Pressure,Boyz II Men

[1667 rows x 2 columns]


In [6]:
options=Options()
driver =webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [7]:
import os
import re
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException

driver.implicitly_wait(5)
url = "https://youtube.com/"
driver.get(url)

checkpoint_file = "url_checkpoint.json"
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
        start_index = checkpoint.get('last_index', 0)
        video_ids_all = checkpoint.get('video_ids_all', [])
        music_ids_all = checkpoint.get('music_ids_all', [])
        titles_all = checkpoint.get('titles_all', [])
        artists_all = checkpoint.get('artists_all', [])
else:
    start_index = 0
    video_ids_all = []
    titles_all = []
    artists_all = []
    music_ids_all = []

# Retry settings
max_retries = 3

for idx, (song_ids, searchWord) in enumerate(zip(Id_keyword['Track ID'][start_index:], Id_keyword['Title Artist'][start_index:])):
    retries = 0
    while retries < max_retries:
        try:
            search_box = driver.find_element(By.XPATH, '//input[@id="search"]')  # Locate the search box element
            search_box.clear()
            search_box.send_keys(searchWord)  # Enter the search term
            search_box.send_keys(Keys.ENTER)  # Press the search button
            time.sleep(2)  # Wait for search results to load
            
            videos = driver.find_elements(By.XPATH, '//*[@id="video-title"]')
            
            if videos:  # If videos are found
                href = videos[0].get_attribute('href')   # Get the first video's URL
                time.sleep(2)
                
                # Extract video link and ID
                pattern = r'(?<=v=)[a-zA-Z0-9_-]+'
                video_id = re.findall(pattern, href)
                video_ids_all.append(','.join(video_id))
                music_ids_all.append(song_ids)
                parts = searchWord.split(",", 1)
                if len(parts) == 2:
                    title, artist = parts
                else:
                    title = parts[0]
                    artist = ""
                titles_all.append(title)
                artists_all.append(artist)
                break  # Successfully handled the search, exit retry loop

            else:
                print(f"No video found for search term: {searchWord}")
                video_ids_all.append("")
                music_ids_all.append(song_ids)
                parts = searchWord.split(",", 1)
                if len(parts) == 2:
                    title, artist = parts
                else:
                    title = parts[0]
                    artist = ""
                titles_all.append(title)
                artists_all.append(artist)
                break  # No videos found, exit retry loop

        except (TimeoutException, WebDriverException) as e:
            retries += 1
            print(f"Error with search term '{searchWord}': {str(e)}. Retrying {retries}/{max_retries}...")
            driver.refresh()  # Refresh the page on network errors
            time.sleep(2)  # Wait for the page to reload

        except Exception as e:
            print(f"Unexpected error with search term '{searchWord}': {str(e)}")
            video_ids_all.append("")
            music_ids_all.append(song_ids)
            parts = searchWord.split(",", 1)
            if len(parts) == 2:
                title, artist = parts
            else:
                title = parts[0]
                artist = ""
            titles_all.append(title)
            artists_all.append(artist)
            break  # Exit retry loop on unexpected errors
    
    # Save checkpoint
    checkpoint = {
        'last_index': idx + 1,  # Save next index
        'video_ids_all': video_ids_all,  # Save lists
        'music_ids_all': music_ids_all,
        'titles_all': titles_all,
        'artists_all': artists_all
    }
    with open(checkpoint_file, 'w') as f:
        json.dump(checkpoint, f)

driver.quit()


Unexpected error with search term 'Do You Wanna Dance,The Mamas & The Papas': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'It Doesn't Have To Be This Way,The Blow Monkeys': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'Ring My Bell,Anita Ward': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'Be Still Now,Luka Bloom': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'You Ought To Be With Me,Al Green': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'You Came Out Of Nowhere,Belinda Carlisle': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'You Used To Hold Me,Calvin Harris': expected string or bytes-like object, got 'NoneType'
Unexpected error with search term 'Love Lovely Love,Jefferson Starship': expected string or bytes-like object, got 'NoneType'

In [8]:
output_1130 = pd.DataFrame({'title':titles_all, 'artist': artists_all, 'Track ID': music_ids_all, 'video_id':video_ids_all})
output_1130.describe()

,title,artist,Track ID,video_id
count,1667,1667,1667,1667
unique,1644,482,1667,1623
top,Without You,Jim Reeves,TRYYDBK128F930FDBC,
freq,3,27,1,37


In [11]:
output_1130.head(10)

,title,artist,Track ID,video_id
0,You Were Alone,Casiotone For The Painfully Alone,TRMMATD128F932C32B,x1t8L44zdIo
1,Without You,Ayo,TRMMKAR128EF3431B1,W6CexnzGj7I
2,Pony Blues,Alvin Youngblood Hart,TRMWQYD128F426639B,tf3HJvLALR0
3,Do You Wanna Dance,The Mamas & The Papas,TRMWXEM128F429D851,
4,Since K Got Over Me,The Clientele,TRMGNHR128F428B17E,u9bHbwywrRc
5,Eres,Alejandro Fernandez,TRMGUNA128F92E4CC0,9nEE-S_tqYo
6,What A Fool Believes,The Doobie Brothers,TRMGYEC128F14A2B40,exnHAdopRxA
7,Desire As,Prefab Sprout,TRMHWLW128F4259742,7oO-fkr25Ts
8,It Doesn't Have To Be This Way,The Blow Monkeys,TRMHFGQ128F4298497,
9,I'm Not Afraid To Die,Gillian Welch,TRMCPBE128F933B0E6,y_ra8X_8waA


In [16]:
pd.set_option('display.max_rows', None)
df = output_1130.copy()
df[df['video_id'].duplicated(keep=False)]




,title,artist,Track ID,video_id
3,Do You Wanna Dance,The Mamas & The Papas,TRMWXEM128F429D851,
8,It Doesn't Have To Be This Way,The Blow Monkeys,TRMHFGQ128F4298497,
30,Ring My Bell,Anita Ward,TRMAKGI128F92E9CE1,
57,Be Still Now,Luka Bloom,TRMDRRB128F4244A73,
74,You Ought To Be With Me,Al Green,TRWGEYM128F933E437,
78,You Came Out Of Nowhere,Belinda Carlisle,TRWCPTG128F1458653,
101,You Used To Hold Me,Calvin Harris,TRWTOIT128F92F88A8,
153,Love Lovely Love,Jefferson Starship,TRGNVAM128F428F119,
201,Sexy Mama,Laura Nyro,TRHCRPS128F4262B64,
222,I'm Going Down,Rose Royce,TRHURXV12903CC4E2A,


In [24]:
manual_id_list = ["xVCxxLW55LU","y_ra8X_8waA","URAqnM1PP5E","4e4742xVdxY","PQCw9FVuZVg"
 ,"QkacvFMoTio","FtNWLvcB5rI","JEvyfzg2DP8","TGU6iy0u7RQ","4J5AoGzjKFo",
 "mZaCrgKNKoM","9srTTnHud8A","12Ossn4zeIk","I_64fZcttGg","8iwBM_YB1sE",
 "YnZgmdEd4-4","v3j9bAVqt3c","LItrToYABIA","XO4msvIA7cw","20S_kwNb4rg",
 "1W5R2NDPaA4","lYU04e3v3Lg","P_2SOYI8AfQ","JXA5soFZ3MI","1TO48Cnl66w",
 "5EO1nfSCKrU","z339NHFROG8","Q3mErFY8XPk","FaYGlE9lHOw","MrA6qZGKFng",
 "dOV5WXISM24","BqTgJL-Aryc","oje_2ED9124","t1pYDgR6zhE","PqpZheCVWlU","",
 "","w2mCKhErn8k","BPexgMDObmA","","Qb1wNA5PnKQ",
 "uCVi3vpGcSg","jsfaRHPdBG8","WyYC9qRlJxY","","DEjy_jQ1p1A",
 "LgdMrvIngAU","_Xl9aHJgrWA","2PD41qBDALE","LdE9gsSiCRo","",
 "","Ui7EZ123-ok"]

df.loc[df['video_id'].duplicated(keep=False), 'video_id'] = manual_id_list


In [28]:
df[df['video_id'].duplicated(keep=False)]
df_real = df[df.video_id != ""]

In [31]:
df_real[df_real['video_id'].duplicated(keep=False)]

,title,artist,Track ID,video_id
8,It Doesn't Have To Be This Way,The Blow Monkeys,TRMHFGQ128F4298497,y_ra8X_8waA
9,I'm Not Afraid To Die,Gillian Welch,TRMCPBE128F933B0E6,y_ra8X_8waA


In [33]:
df_real.loc[df_real['video_id'].duplicated(keep=False),'video_id']=["y_ra8X_8waA","LATJglJUyZ4"]

In [34]:
df_real[df_real['video_id'].duplicated(keep=False)]

,title,artist,Track ID,video_id


In [37]:
df_real.to_csv("youtubeUrl_241130.csv",index=False)

In [36]:
df_real.describe()

,title,artist,Track ID,video_id
count,1661,1661,1661,1661
unique,1639,482,1661,1661
top,Without You,Jim Reeves,TRYYDBK128F930FDBC,01pGEs7EMIo
freq,3,27,1,1


In [39]:
sim =pd.read_csv("../data/similarity_results.csv")
sim.head()

,Track ID,Title,Artist,Lyrics,Genre,Group Index
0,TRGUZSY128F9312BF9,Hockey Skates,Kathleen Edwards,[Verse 1]\r\nGoing down in the same old town\r...,folk,0
1,TRVHERE128F930FD80,Southern Rain,Cowboy Junkies,The flies have quit their buzzing\r\nEven Bear...,folk,0
2,TRIMTDE128F92F740A,Somewhere Else,Kathleen Edwards,I used to look for you out in the daylight\r\n...,folk,0
3,TRVQNRM128F145AB61,Crying,J.J. Cale,"Every time I see you, all that you do\r\nCryin...",classic pop and rock,0
4,TRCYJRW128F92E7446,Ol' Cook Pot,The Duhks,Mama gonna bargain with the 'ol cook pot\r\nTr...,folk,0


In [40]:
sim["Genre"].value_counts()

Genre
classic pop and rock     1167
folk                      551
soul and reggae           137
metal                      87
dance and electronica      68
pop                        56
jazz and blues             36
punk                       29
classical                   2
hip-hop                     1
Name: count, dtype: int64

In [45]:
filtered = sim[~sim["Genre"].isin(['classical', 'hip-hop'])]
filtered.to_csv("data/sim_except_HC.csv",index=False)